# Substitution Ciphers

In [ ]:
# import modules
import matplotlib.pyplot as plt
import pickle

## Introduction

A typical scenario in data security involves transmitting **private information** from a sender to a receiver through a public space. This information is kept secret thanks to Encryption and Decryption functions that work with shared information, known as a **key**. The goals of data security include *confidentiality* or privacy (ensuring any eavesdropper cannot understand the information), *integrity*, *authentication*, and *non-repudiation*.

A substitution cipher is *a symmetric encryption method* where each letter in the plaintext is replaced by another letter. The replacement is determined by a fixed system, such as shifting letters by a certain number of positions in the alphabet (as in the **Caesar cipher**), or by a mapping of letters (as in the **simple substitution cipher**). Despite being simple to implement, these ciphers are **vulnerable** to frequency analysis attacks, where the frequency of letters in the ciphertext is compared to the known frequency of letters in the language used in the text.


## Caesar Cipher

The Caesar Cipher is one of the simplest and historically known encryption techniques. It’s named after Roman Emperor Julius Caesar, who used it in his private and military correspondence. The goal is to **encrypt text**, not bits. The idea is that each letter of the plaintext (original text) is replaced with a letter some fixed number of positions down the alphabet. *For example, with a shift of 4, H would be replaced by L, E would become I, and so on*. Because the replacement remains the same throughout the message, the **key** is only the **shift value** of the alphabet. To decrypt the message, you simply need to shift each letter back by the same number of positions (*symmetric algorithm*). [[1]](#[1])

The advantage of simplicity in implementation is paid with vulnerability. Indeed, there are some ways to break it:
* **Brute Force Attack**: Given the Caesar Cipher’s limited key space (26 possible shifts in English), an attacker can systematically try all possible shifts to decrypt the message. This method is guaranteed to succeed due to the finite nature of possible keys.
* **Frequency Analysis Attack**: This attack leverages the fact that each language has a distinctive letter frequency. By comparing the frequencies of characters in the encrypted message to those in the language, an attacker can guess the shift value.

### Encryption

The encryption function takes each plaintext character and, thanks to a mapping between plaintext and ciphertext internally created from the given key as shift value, they will be replaced to return a ciphertext; spaces or special characters will not be substituted.
<br><br>As shown in the example below, a *shift of 4* will create this mapping which allows *‘hello!’* plaintext to be encrypted as *‘lips!’*.
$$Plain\; -ABCDEFGHIJKLMNOPQRSTUVWXYZ$$$$Ciphe\; -EFGHIJKLMNOPQRSTUVWXYZABCD$$

In [ ]:
def caesar_encrypt(plaintext, shift=0):
    # Help Description
    ''' Encrypt `plaintext` (str) as a caesar cipher with a given `shift` (int) 
    '''
    # Define dictionary {eng_letter : shift_letter}
    unicode_chr = [i for i in range(ord('a'), ord('z')+1)]

    engl_shiftl = {}
    for i,k in enumerate(unicode_chr):
        engl_shiftl[chr(k)] = chr(unicode_chr[(i+shift)%26])

    # Ciphertext
    ciphertext = ''
    for l in plaintext:
        # Get value from dictionary giving plaintext
        ciphertext += engl_shiftl.setdefault(l, l)
    
    return ciphertext

In [ ]:
# code snippet to test the implementation of the encryption function
plaintext = 'hello!' 
ciphertext = caesar_encrypt(plaintext, shift=4)

print(plaintext, '->', ciphertext) # expected output 'hello! -> lipps!'

### Decryption

Just like encryption, the decryption function works in reverse: thanks to a mapping, in this case between ciphertext and plaintext, created from the same given key, each ciphertext character will be replaced to return the plaintext. Again, the same applies for spaces and special characters.

In [ ]:
def caesar_decrypt(ciphertext, shift=0):
    ''' Decrypt `ciphertext` (str) as a caesar cipher with a given `shift` (int) 
    '''
    # Define dictionary {shift_letter : eng_letter}
    unicode_chr = [i for i in range(ord('a'), ord('z')+1)]
    
    shiftl_engl = {}
    for i,k in enumerate(unicode_chr):
        shiftl_engl[chr(unicode_chr[(i+shift)%26])] = chr(k)
        
    # Plaintext
    plaintext = ''
    for l in ciphertext:
        # Get value from dictionary giving ciphertext
        plaintext += shiftl_engl.setdefault(l, l)
            
    return plaintext

In [ ]:
# code snippet to test the implementation of the decryption function
ciphertext = 'lipps!' # 'hello!' encoded with shift=4
plaintext = caesar_decrypt(ciphertext, shift=4)

print(ciphertext, '->', plaintext)  # expected output 'lipps! -> hello!'

### Ciphertext

In [ ]:
# Load ciphertext
with open(file='ciphertext_caesar.txt', mode='r') as f:
    ctxt = f.read()

print(ctxt[0:427] + '...')

### Brute Force Attack

*Assuming you have a finite number of possible keys*, which is the maximum number of letters in that language, the Caesar cipher becomes vulnerable to a brute force attack. In this case (English), since the key space is 26 (actually **25** because a shift value equal to zero isn’t counted), a brute force attack is possible. Therefore, any eavesdroppers can try all possible keys and find the correct shift value, which allows them to decrypt the message and understand the information. [[1]](#[1])

In [ ]:
# Try all possible key
force_attack = []

for i in range(1,26): #shift = 0 is the same as ctxt
    force_attack.append(caesar_decrypt(ctxt,i))

In [ ]:
# select the right shift
rshift = 20
# decrypt ciphertext
print('The right key as shift value is: ', rshift, '\n\n', force_attack[rshift-1])

## Simple Substitution Cipher

Simple substitution is another *symmetric encryption method* which has the goal to **encrypt text** and not bits. The idea in this case is to replace each letter of plaintext with another letter of the alphabet, hence the **key** is a **bijective mapping** between alphabet letters. For example, key mapping could be *{qwertyuiopasdfghjklzxcvbnm}*, in this case *A would be replaced by Q, B would become W, and so on*; if the mapping is equal to the original alphabet shifted, it is like having a Caesar Chiper. [[2]](#[2])
<br>The receiver deciphers the text by performing the inverse substitution process to extract the original message.

The previous assumption of having a finite number of possible keys is no more valid, so a *brute force attack is unfeasible*: even assuming an eavesdropper can try one key every 1ns, key space for English alphabet is equal to 26! ~ 10$^{27}$, it would take 10 billion years to try all keys.
<br>Simple substitution remains a historical cipher that is **unsecure**, so its major weakness is that each letter of a plaintext is always mapped to the same letter of the ciphertext hence statistical properties of the plaintext are preserved in the ciphertext. Those are the key points for  **Frequency Analysis Attack**.

### Encryption

Sender has to compute the encryption of the plaintext before transmission. Thanks to the given mapping, the encryption takes each letter from plaintext and substitutes them with the ciphertext letters; no substitution for space and special characters.

In [ ]:
def substitution_encrypt(plaintext, mapping):
    ''' Encrypt `plaintext` (str) as a simple substitution cipher with a given 
        `mapping` (dict) from plaintext letters to ciphertext letters '''

    # Ciphertext
    ciphertext = ''
    for l in plaintext:
        # Get value from dictionary giving plaintext
        ciphertext += mapping.setdefault(l, l)
            
    return ciphertext

In [ ]:
# code snippet to test the implementation  of the encryption function
plaintext = 'hello!'
mapping = {'h': 'a', 'e': 'p', 'l': 'w', 'o': 'q'}

ciphertext = substitution_encrypt(plaintext, mapping)

print(plaintext, '->', ciphertext) # expected output 'hello! -> apwwq!'

### Decryption

The receiver has to compute the decryption of the ciphertext to extract the information from the message. In this case, it needs the key mapping, but from ciphertext to plaintext letters. Since simple substitution is a symmetric algorithm, or the key is shared information, the given key mapping is from plaintext to ciphertext letters. So, after the inversion of the mapping, the decrypter is able to extract the plaintext with substitution.

In [ ]:
def substitution_decrypt(ciphertext, mapping):
    ''' Decrypt `ciphertext` (str) as a simple substitution cipher with a given 
       `mapping` (dict) from plaintext letters to ciphertext letters '''

    # Invert mapping
    imapping = {}
    for k,v in mapping.items():
        imapping[v] = k
    
    # Plaintext
    plaintext = ''
    for l in ciphertext:
        # Get value from dictionary inverted giving ciphertext
        plaintext += imapping.setdefault(l, l)
            
    return plaintext

In [ ]:
# code snippet to test the implementation of the decryption function
mapping = {'h': 'a', 'e': 'p', 'l': 'w', 'o': 'q'}  # previous mapping
ciphertext = 'apwwq!'

plaintext = substitution_decrypt(ciphertext, mapping)

print(ciphertext, '->', plaintext)  # expected output 'apwwq! -> hello!'

### Ciphertext

In [ ]:
# Load ciphertext
with open(file='ciphertext_simple.txt', mode='r') as f:
    ctxt = f.read()

print(ctxt[0:414] + '...')

### Frequency Analysis Attack

Determine the frequency of the letters in the ciphertext and match it with that of the plaintext; a distribution taken from a large text can be used as a model of plaintext letter frequency. Thus, *match the most frequent letter in the ciphertext with the one in the model, then the second most frequent, and so on*. Having the mapping, it’s possible to try to decrypt the text. Certainly, it will not entirely decrypt because probably there are some distributions of letters which are more or less at the same value, hence they could be mistakenly exchanged. At this point, it’s possible to identify some words that are arising. By comparing the word that is believed to be in the plaintext and the same encrypted word, it is possible to rewrite the mapping while *maintaining the bijective property* of the function. Repeating the process many times, the final mapping is eventually reached and the plaintext is correctly decrypted. [[3]](#[3])

#### English Letters Distribution

In [ ]:
# function to infer the letters distribution from a text
def letter_distribution(text):
    ''' Return the `distribution` (dict) of the letters in `text` (str) '''
    
    # Initialize a dictionary {letter : freq}
    distribution = {chr(i): 0 for i in range(ord('a'), ord('z')+1)}

    # Find length of text counting only alphabet letters
    len_alpha = len([ele for ele in text if ele.isalpha()])

    # Set frequency for all letter in dictionary
    for i in distribution.keys():
        distribution[i] = text.count(i)/len_alpha
        
    return distribution   

In [ ]:
# code snippet to test the implementation of `letter_distribution`
text = 'hello world!'
dist = letter_distribution(text)

print({k:v for k,v in dist.items() if v != 0})

# expected ouput: 
# {'d': 0.1, 'e': 0.1, 'h': 0.1, 'l': 0.3, 'o': 0.2, 'r': 0.1, 'w': 0.1, ...}

In [ ]:
# load text used as model of plaintext letters frequency
with open(file='The-Adventure-of-the-Dancing-Men.txt', encoding='utf-8') as f:
    txt = f.read()

print(txt[0:414] + '...')

In [ ]:
# estimate the English letters distribution 
dist_pattern = letter_distribution(txt.lower())

In [ ]:
# plot the English letter distribution
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.bar(dist_pattern.keys(), dist_pattern.values(), align="center", width=0.6, alpha=0.9)
_ = ax.set(ylabel='Frequency', title='Letters Distribution of "The-Adventure-of-the-Dancing-Men"')

In [ ]:
# store the distribution as a pickle file
with open(file='Distribution_Pattern.pkl', mode='wb') as f:
    pickle.dump(dist_pattern, f)  

#### Perform attack

In [ ]:
# estimate the ciphertext letters distribution
dist = letter_distribution(ctxt)

# plot the ciphertext letters distribution
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.bar(dist.keys(), dist.values(), align="center", width=0.6, alpha=0.9)
_ = ax.set(ylabel='Frequency', title='Letters Distribution of Simple Substitution ciphertext')

In [ ]:
# load distribution model
with open(file='Distribution_Pattern.pkl', mode='rb') as f:
    dist_pattern = pickle.load(f)
    
# sort distributions by value
key = [k[0] for k in sorted(dist_pattern.items(), key=lambda x:x[1])]
value = [v[0] for v in sorted(dist.items(), key=lambda x:x[1])]

# create mapping from matching of the two sorted distribution
mapping = {k:v for k,v in sorted(zip(key,value))}

In [ ]:
# print mapping
print('Plain  - ',' '.join(mapping.keys()))
print('Chiper - ',' '.join(i[1] for i in mapping.items()))

In [ ]:
# print decrypted plaintext
decrypt_1 = substitution_decrypt(ctxt, mapping)
print(decrypt_1[0:2201]) # Less character for simplicity

Like has been explained before, with the matching of the two distributions, *the mapping has some letters that are mistakenly exchanged*, so as seen above, the text is **not correctly decrypted**. But now, you could search if there are some words that are almost correctly decrypted. Considering these words that are believed to be in the plaintext and matching them with the cipher words at the same position, it is possible to rewrite the mapping and observe if the text decryption works properly, a little better, or if the new mapping doesn’t work well.

In [ ]:
# function to modify mapping maintaining bijective property 
def swap_mapping(mapping, mapping_ToInsert):
    ''' Return `new_mapping` (dict) from previous `mapping` (dict) by adding the new key-value couples that 
    are present in `mapping_ToInsert` maintaining bijective property, or 26 couples at all'''
    
    new_mapping = mapping.copy()

    for k,v in mapping_ToInsert.items():
        # add new KEY-value
        new_mapping[k.upper()] = v
        # search for the old key that holds the new value and assign it the old value that was held by the new key
        new_mapping[[i for i in new_mapping.keys() if new_mapping[i]==v][0]] = new_mapping[k]
        # delete old key-value
        del new_mapping[k]
        # sorted by key
        new_mapping = {k:v for k,v in sorted(new_mapping.items())}
        
    return new_mapping

For example, pausing at the word that starts at index *1939*  seems to be **"example"**. So the first four letters are properly mapped, you could try to write mapping again by observing the cipher word at the same position.

In [ ]:
# index
p = 1939
# print word
print('Considering the first decryption trial "'+decrypt_1[p:p+7]+'", the cipher word "'+ctxt[p:p+7]+'" should decrypt to "example".\n')

# print mapping_ToInsert
print('Plain  - ', decrypt_1[p:p+7], '---> examPLe ---> P L')
print('Chiper - ', ctxt[p:p+7], '--->', ctxt[p:p+7], '--->', ctxt[p+4], ctxt[p+5])

In [ ]:
# Rewrite mapping
example_mapping = swap_mapping(mapping, {'p':'k', 'l':'i'})
# print mapping
print('Plain  - ',' '.join(example_mapping.keys()))
print('Chiper - ',' '.join(i[1] for i in example_mapping.items()))

In [ ]:
# print decrypted plaintext
decrypt_2 = substitution_decrypt(ctxt, example_mapping)
print(decrypt_2[0:2201])

A very little better decryption because now you can focus with cunning on date, in particular as date before number there is for sure a month. Focusing on first date **"aPsiL 30"**, taking on accuracy of the previous mapping, the word of plaintext can only be **"april"**. So, you can repeat the process again assigning the only letter that aren't properly mapping and watching again the word of ciphertext at same position.

In [ ]:
# index
p = 23
# print word
print('Considering the previous decryption trial "'+decrypt_2[p:p+5]+'", the cipher word "'+ctxt[p:p+5]+'" should decrypt to "april".\n')

# print mapping_ToInsert
print('Plain  - ', decrypt_2[p:p+5], '---> apRil ---> R')
print('Chiper - ', ctxt[p:p+5], '--->', ctxt[p:p+5], '--->', ctxt[p+2])

In [ ]:
# Rewrite mapping
april_mapping = swap_mapping(example_mapping, {'r':'w'})
# print mapping
print('Plain  - ',' '.join(april_mapping.keys()))
print('Chiper - ',' '.join(i[1] for i in april_mapping.items()))

In [ ]:
# print decrypted plaintext
decrypt_3 = substitution_decrypt(ctxt, april_mapping)
print(decrypt_3[0:2201])

Now with the second date **"cebRwaRg"** and remember the assumption that *'e'* and *'a'* are properly mapped (*'ExAmplE'*) you can get the right word of plaintext. 

In [ ]:
# index
p = 38
# print word
print('Considering the previous decryption trial "'+decrypt_3[p:p+8]+'", the cipher word "'+ctxt[p:p+8]+'" should decrypt to "february".\n')

# print mapping_ToInsert
print('Plain  - ', decrypt_3[p:p+8], '---> february ---> F U Y')
print('Chiper - ', ctxt[p:p+8], '--->', ctxt[p:p+8], '--->', ctxt[p], ctxt[p+4], ctxt[p+7])

In [ ]:
# Rewrite mapping
february_mapping = swap_mapping(april_mapping, {'f':'d', 'u':'o', 'y':'q'})
# print mapping
print('Plain  - ',' '.join(february_mapping.keys()))
print('Chiper - ',' '.join(i[1] for i in february_mapping.items()))

In [ ]:
# print decrypted plaintext
decrypt_4 = substitution_decrypt(ctxt, february_mapping)
print(decrypt_4[0:2201])

Once understood the trick game become easy to play.

In [ ]:
# index
p = 64
# print word
print('Considering the previous decryption trial "'+decrypt_4[p:p+8]+'", the cipher word "'+ctxt[p:p+8]+'" should decrypt to "american".\n')

# print mapping_ToInsert
print('Plain  - ', decrypt_3[p:p+8], '---> american ---> C N')
print('Chiper - ', ctxt[p:p+8], '--->', ctxt[p:p+8], '--->', ctxt[p+5], ctxt[p+7])

In [ ]:
# Rewrite mapping
american_mapping = swap_mapping(february_mapping, {'c':'g', 'n':'b'})
# print mapping
print('Plain  - ',' '.join(american_mapping.keys()))
print('Chiper - ',' '.join(i[1] for i in american_mapping.items()))

In [ ]:
# print decrypted plaintext
decrypt_5 = substitution_decrypt(ctxt, american_mapping)
print(decrypt_5[0:5984])

In [ ]:
# index and length
index_len = ((3828, 8, 'analyzer'),
             (0, 6, 'claude'),
             (109, 8, 'computer'),
             (132, 13, 'cryptographer'),
             (239, 19, 'electronic circuits'),
             (88, 19, 'electrical engineer'),
             (170, 11, 'information'),
             (146, 5, 'known'),
             (2598, 10, 'new jersey'))

# print mapping_ToInsert
for i,l,s in index_len:
    # print word
    print('Considering the previous decryption trial "'+decrypt_5[i:i+l]+'", the cipher word "'+ctxt[i:i+l]+'" should decrypt to "'+s+ '".\n')
    # print mapping_ToInsert
    print('Plain  - ', decrypt_5[i:i+l], '--->', s, ' --->', [kk.upper()+' ' for kk in s])
    print('Chiper - ', ctxt[i:i+l], '--->', ctxt[i:i+l], '--->', [ctxt[i+kk]+' ' for kk in range(l)])
    print()

In [ ]:
# Rewrite mapping
last_mapping = swap_mapping(american_mapping, {'z':'y', 'd':'a', 'o':'l', 't':'s', 'g':'e', 'h':'v', 's':'n', 'w':'j', 'j':'u'})

# print mapping
print('The right mapping.\n')
print('Plain  - ',' '.join(last_mapping.keys()))
print('Chiper - ',' '.join(i[1] for i in last_mapping.items()))

In [ ]:
# print decrypted plaintext
print(substitution_decrypt(ctxt, last_mapping))

## Vigenère Cipher

**Polyalphabetic** substitution cipher, the Vigenère cipher is a method of encrypting alphabetic **text** where each letter of the plaintext is encoded with a different **Caesar cipher**, *whose increment is determined by the corresponding letter of another text: the key*. 

Just as you saw with the Simple Substitution Cipher, the brute force attack doesn’t work because without knowing the key length, it is unfeasible to try each single combination. Even if the key length is known, it is impossible to find it. In any case, the way to break this algorithm is again based on **frequency**, but it is more complicated and the longer the key, the greater the difficulty. [[4]](#[4])

### Encryption

For Vigenère encryption, each alphabetic character of the plaintext will be encrypted individually by the Caesar Cipher algorithm. Assuming key characters as a number, where in your case (English) ‘a’ is equal to 0 and ‘z’ is equal to 25, and so on for the intermediate values, you can easily obtain the shift value. When the last character of the key is used, the new plaintext alphabetic character will simply be encrypted with the first key character again. So, for example, if the key length is 5, then every fifth alphabetic character of the plaintext will be encrypted with the same shift value for Caesar Cipher encryption.

In [ ]:
def vigenere_encrypt(plaintext, key):
    ''' Encrypt `plaintext` (str) as a Vigenère cipher with a given `key` (str) '''
    
    # lower key to get correctly shift value
    key = key.lower()
    
    # Ciphertext
    ciphertext = ''
    i_key = 0 #key index
    
    for i in plaintext:
        # get shift from key character
        shift = ord(key[i_key%(len(key))])-97
        # caesar encryption of plaintext character
        ciphertext += caesar_encrypt(i, shift)
        if i.isalpha():
            # increment key index
            i_key += 1
            
    return ciphertext

In [ ]:
# code snippet to test the implementation  of the encryption function
plaintext = 'hello!'
key = 'key'

ciphertext = vigenere_encrypt(plaintext, key)

print(plaintext, '->', ciphertext) # expected output 'hello! -> rijvs!'

### Decryption

Decryption works as the inverse of encryption, as simple as that. Indeed, in the same way as above, the shift value is computed from the key text, and then to get the plaintext, the function will simply apply the Caesar decryption.

In [ ]:
def vigenere_decrypt(ciphertext, key):
    ''' Decrypt `ciphertext` (str) as a Vigenère cipher with a given `key` (str) '''
    
    # lower key
    key = key.lower()
    
    # plaintext
    plaintext = ''
    i_key = 0
    
    for i in ciphertext:
        shift = ord(key[i_key%(len(key))])-97
        # caesar decryption of ciphertext character
        plaintext += caesar_decrypt(i, shift)
        if i.isalpha():
            i_key += 1
            
    return plaintext

In [ ]:
# code snippet to test the implementation  of the encryption function
ciphertext = 'rijvs!'
key = 'key'

plaintext = vigenere_decrypt(ciphertext, key)

print(ciphertext, '->', plaintext) # expected output 'rijvs! -> hello!'

### Ciphertext

In [ ]:
# Load ciphertext
with open(file='ciphertext_vigenere.txt', mode='r') as f:
    ctxt = f.read()

print(ctxt[0:917] + '...')

### Attack

Attack is a combination of **frequency analysis** and **examination of repeating** groups of letters in the ciphertext. In general, with the examination of repeating groups of letters, the lengths of the gaps between these repeating sequences are likely to be multiples of the key length. So, by finding the greatest common divisor of these lengths, you can get a good estimate of the key length[[5]](#[5])[[6]](#[6]). Once you know the key length, you can perform a frequency analysis on each set of letters encrypted with the same letter of the key[[3]](#[3])[[6]](#[6]). This time, the key length comes with the assignment and is equal to 6.

In [ ]:
key_len = 6
key = ''

# load distribution model
with open(file='Distribution_Pattern.pkl', mode='rb') as f:
    dist_pattern = pickle.load(f)

for i in range(key_len):
    # take letters of ctxt of every i+len(key)
    cctxt = ''.join([l for l in ctxt if l.isalpha()][i::key_len])
    # compute distribution 
    distribution = letter_distribution(cctxt)
    # get most frequent letter
    mFreq_clett = sorted(distribution.items(), key=lambda x:x[1], reverse=True)[0][0]
    mFreq_plett = sorted(dist_pattern.items(), key=lambda x:x[1], reverse=True)[0][0]
    # calculate key char as shift from the two most frequent letter
    key += chr((ord(mFreq_clett) - ord(mFreq_plett))%26 + 97)

print('The right key is: "'+key+'".')

In [ ]:
# print decrypted text
print(vigenere_decrypt(ctxt, key))

## Conclusion

As one considers the encryption techniques covered, it is clear that although algorithms such as the Caesar Cipher and Simple Substitution Cipher are very simple and thus facilitate simple encryption and decryption procedures, their very simplicity also makes them extremely susceptible to cryptanalysis. Due to its small key space, the Caesar Cipher is especially vulnerable to brute force attacks since it is simple to cycle through every conceivable shift. Even with a wider key space, the Simple Substitution Cipher cannot hide the letter frequency, leaving it vulnerable to frequency analysis. The procedure becomes slightly more complex using Vigenère, but it is still breakable unless the key is as lengthy as plaintext. These flaws demonstrate the need for more complex encryption methods in order to safeguard data from these ever-evolving tactics. The exploration of these algorithms serves as a reminder of the continuous evolution required in the field of cryptography.

## Reference

<span id='[1]' > [1] [Caesar Cipher](https://en.wikipedia.org/wiki/Caesar_cipher) </span>
<br><span id='[2]' > [2] [Simple Substitution Cipher](https://simple.wikipedia.org/wiki/Substitution_cipher) </span>
<br><span id='[3]' > [3] [Frequency Analysis attack](https://en.wikipedia.org/wiki/Frequency_analysis) </span>
<br><span id='[4]' > [4] [Vigenère Cipher](https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher) </span>
<br><span id='[5]' > [5] [Kasiski's Method](https://pages.mtu.edu/~shene/NSF-4/Tutorial/VIG/Vig-Kasiski.html) </span>
<br><span id='[6]' > [6] "[Cryptanalysis of the Vigenère Cipher: Kasiski Test](https://www.nku.edu/~christensen/1402%20vigenere%20cryptanalysis.pdf)", (2015) Chris Christensen.</span>